In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/QuMaDe2/fgImages'

In [ ]:
#pip install 'h5py==2.10.0' --force-reinstall

## Loading the model for generating Segmentation images

In [ ]:
import cv2
im = cv2.imread("fgbg000000.jpg")

In [ ]:
# Used for segmentation

# Tensorflow
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

# I/O libraries
import os
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

# Helper libraries
import matplotlib
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import cv2 as cv
from tqdm import tqdm
import IPython
from sklearn.metrics import confusion_matrix
from tabulate import tabulate

# Comment this out if you want to see Deprecation warnings
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

TensorFlow 1.x selected.
1.15.2


In [ ]:
#Used for segmentation

class DeepLabModel(object):
    """Class to load deeplab model and run inference."""

    FROZEN_GRAPH_NAME = 'frozen_inference_graph'

    def __init__(self, tarball_path):
        """Creates and loads pretrained deeplab model."""
        self.graph = tf.Graph()
        graph_def = None

        # Extract frozen graph from tar archive.
        tar_file = tarfile.open(tarball_path)
        for tar_info in tar_file.getmembers():
            if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
                file_handle = tar_file.extractfile(tar_info)
                graph_def = tf.GraphDef.FromString(file_handle.read())
                break
        tar_file.close()

        if graph_def is None:
            raise RuntimeError('Cannot find inference graph in tar archive.')

        with self.graph.as_default():
            tf.import_graph_def(graph_def, name='')
        self.sess = tf.Session(graph=self.graph)

    def run(self, image, INPUT_TENSOR_NAME = 'ImageTensor:0', OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'):
        """Runs inference on a single image.

        Args:
            image: A PIL.Image object, raw input image.
            INPUT_TENSOR_NAME: The name of input tensor, default to ImageTensor.
            OUTPUT_TENSOR_NAME: The name of output tensor, default to SemanticPredictions.

        Returns:
            resized_image: RGB image resized from original input image.
            seg_map: Segmentation map of `resized_image`.
        """
        width, height = image.size
        target_size = (2049,1025)  # size of Cityscapes images
        resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
        batch_seg_map = self.sess.run(
            OUTPUT_TENSOR_NAME,
            feed_dict={INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
        seg_map = batch_seg_map[0]  # expected batch size = 1
        if len(seg_map.shape) == 2:
            seg_map = np.expand_dims(seg_map,-1)  # need an extra dimension for cv.resize
        seg_map = cv.resize(seg_map, (width,height), interpolation=cv.INTER_NEAREST)
        return seg_map

In [ ]:
#Used for segmentation

def create_label_colormap():
    """Creates a label colormap used in Cityscapes segmentation benchmark.

    Returns:
        A Colormap for visualizing segmentation results.
    """
    colormap = np.array([
        [128,  64, 128],
        [244,  35, 232],
        [ 70,  70,  70],
        [102, 102, 156],
        [190, 153, 153],
        [153, 153, 153],
        [250, 170,  30],
        [220, 220,   0],
        [107, 142,  35],
        [152, 251, 152],
        [ 70, 130, 180],
        [220,  20,  60],
        [255,   0,   0],
        [  0,   0, 142],
        [  0,   0,  70],
        [  0,  60, 100],
        [  0,  80, 100],
        [  0,   0, 230],
        [119,  11,  32],
        [  0,   0,   0]], dtype=np.uint8)
    return colormap


def label_to_color_image(label):
    """Adds color defined by the dataset colormap to the label.

    Args:
        label: A 2D array with integer type, storing the segmentation label.

    Returns:
        result: A 2D array with floating type. The element of the array
            is the color indexed by the corresponding element in the input label
            to the PASCAL color map.

    Raises:
        ValueError: If label is not of rank 2 or its value is larger than color
            map maximum entry.
    """
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = create_label_colormap()

    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')

    return colormap[label]


def vis_segmentation(image, seg_map):
    """Visualizes input image, segmentation map and overlay view."""
    plt.figure(figsize=(20, 4))
    grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

    plt.subplot(grid_spec[0])
    plt.imshow(image)
    plt.axis('off')
    plt.title('input image')

    plt.subplot(grid_spec[1])
    seg_image = label_to_color_image(seg_map).astype(np.uint8)
    plt.imshow(seg_image)
    plt.axis('off')
    plt.title('segmentation map')

    plt.subplot(grid_spec[2])
    plt.imshow(image)
    plt.imshow(seg_image, alpha=0.7)
    plt.axis('off')
    plt.title('segmentation overlay')

    unique_labels = np.unique(seg_map)
    ax = plt.subplot(grid_spec[3])
    plt.imshow(FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
    ax.yaxis.tick_right()
    plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
    plt.xticks([], [])
    ax.tick_params(width=0.0)
    plt.grid('off')
    plt.show()


LABEL_NAMES = np.asarray([
    'road', 'sidewalk', 'building', 'wall', 'fence', 'pole', 'traffic light',
    'traffic sign', 'vegetation', 'terrain', 'sky', 'person', 'rider', 'car', 'truck',
    'bus', 'train', 'motorcycle', 'bicycle', 'void'])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

In [ ]:
#Used for segmentation

MODEL_NAME = 'mobilenetv2_coco_cityscapes_trainfine'
#MODEL_NAME = 'xception65_cityscapes_trainfine'

_DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
_MODEL_URLS = {
    'mobilenetv2_coco_cityscapes_trainfine':
        'deeplabv3_mnv2_cityscapes_train_2018_02_05.tar.gz',
    'xception65_cityscapes_trainfine':
        'deeplabv3_cityscapes_train_2018_02_06.tar.gz',
}
_TARBALL_NAME = 'deeplab_model.tar.gz'

model_dir = tempfile.mkdtemp()
tf.gfile.MakeDirs(model_dir)

download_path = os.path.join(model_dir, _TARBALL_NAME)
print('downloading model, this might take a while...')
urllib.request.urlretrieve(_DOWNLOAD_URL_PREFIX + _MODEL_URLS[MODEL_NAME], download_path)
print('download completed! loading DeepLab model...')

MODEL = DeepLabModel(download_path)
print('model loaded successfully!')

downloading model, this might take a while...
download completed! loading DeepLab model...
model loaded successfully!


In [ ]:
#Used for segmentation

# This function returns .npy format for the segmentation map of the given background image
def generatesegments(bg):
  #print("generating segmentation images")
  original_im = bg
  if(type(bg)==type("s")):
    original_im = Image.open(bg)
  seg_map = MODEL.run(original_im)
  seg_image = Image.fromarray(label_to_color_image(seg_map).astype(np.uint8), mode='RGB')
  seg_image.save("seg.jpg")
  np.save("temp_seg.npy", seg_map)
  seg = np.load("temp_seg.npy")
  return seg

## Flipping the foreground images

In [ ]:
def flipforeground(fgpath, imgno):
  fglist = list(filter(lambda x: x.endswith('png'), os.listdir(fgpath)))
  print('Creating flipped foregrounds')
  for i in range(len(fglist)):
    fgimg = os.path.join(fgpath, fglist[i])
    fgim = Image.open(fgimg)
    fgout = fgim.transpose(Image.FLIP_LEFT_RIGHT)
    fgout.save(os.path.join(fgpath, f"image{imgno:0>3}.png"))
    imgno +=1

#We have 100 foreground images. the flipped images numbers will start from 101
#flipforeground('/content/drive/MyDrive/QuMaDe2/fgImages', imgno=101)

## Generating the masks for foreground images

In [ ]:
# #foreground image names are in the pattern image001.png... want to save this as mask001.jpg
# from PIL import Image
# import numpy as np
# import osqp
# fgpath = os.path.join('/content/drive/MyDrive/QuMaDe2', 'fgImages')
# for f in tqdm(os.listdir(fgpath)):
#   rgba = np.array(Image.open(fgpath+'/'+f).convert('RGBA'))
#   rgba[rgba[..., -1]>0] = [255,255,255,255]
#   rgba[rgba[..., -1]==0] = [0, 0,0, 255]
#   im = (Image.fromarray(rgba)).convert('RGB')
#   im.save('/content/drive/MyDrive/QuMaDe2/masks/mask'+f[3:-4]+'.jpg')

## Loading the model and cooking necessary functions for Depth map predictions

In [ ]:
from random import seed
import random
# seed random number generator

In [ ]:
import math
import os
from tqdm.notebook import tqdm

def randomflip(im):
  r = random.random()
  if r>0.5:
    return im.transpose(Image.FLIP_LEFT_RIGHT), True
  return im, False

# crops image and its segment to given size and possibly flips is horizontally
def random_crop_n_flip(img,segment, newsize=(250, 250)):
  img, f = randomflip(img)
  w, h = img.size

  if h>w:
    w, h = (newsize[0], int(newsize[0]*h/w))
    img = img.resize((w,h))
    #segment = generatesegments(img)
    left, right = 0, w
    bottom, top = h, h-w
  else:
    w, h = (int(newsize[1]*w/h), newsize[1])
    img = img.resize((w,h))
    #segment = generatesegments(img)
    left = int(random.random()*(w-h))
    right = left+h
    bottom, top = h, 0

  cropped = img.crop((left, top, right, bottom))  # size: 45, 45
  #cs = np.flip(segment, 1) if f else segment
  #cs1 = cs[top:bottom, left:right]
  #print(w,h,top,bottom,left,right,cs.shape,cs1.shape,cropped.size)
  return cropped  #, cs1

In [ ]:
# def area(a, x, y, limit, cur):
#   if cur > limit:
#     return
#   a[x][y]=2
#   cur += 1
#   if a[x-1][y]==1:
#     cur = area(a, x-1, y, limit, cur)
#   if a[x+1][y]==1:
#     cur = area(a, x+1, y, limit, cur)
#   if a[x][y-1]==1:
#     cur = area(a, x, y-1, limit, cur)
#   if a[x][y+1]==1:
#     cur = area(a, x, y+1, limit, cur)
#   return cur

def processrow(a, r, limit):
  h, w = a.shape
  i = 0
  while i<w:
    while i<w and a[r][i]==0:
      i+=1
    j = i
    while i<w and a[r][i]>0:
      i+=1
    if i-j < limit:
      while j<i:
        a[r][j] = 0
        j += 1

def processcol(a, c, limit):
  h, w = a.shape
  i = 0
  while i<h:
    while i<h and a[i][c]==0:
      i+=1
    j = i
    while i<h and a[i][c]>0:
      i+=1
    if i-j < limit:
      while j<i:
        a[j][c] = 0
        j += 1

def boundaryreduce(a, limit=5):
  h,w = a.shape
  for i in range(h):
    processrow(a, i, limit)
  for i in range(w):
    processcol(a, i, limit)

In [ ]:
def show_binary_image(img):
  display(Image.fromarray((img*255).astype(np.uint8), mode='L'))

def extract_ground(segment, skylimit):
  # get all rows having road, sidewalk or terrain below the sky
  g = ((segment==0) | (segment==1) | (segment==9))*1
  # process g such that small patches and ground above skylimit are removed.
  g[:skylimit, :] = 0
  boundaryreduce(g, 10)
  #show_binary_image(g)
  return g

def pick_random_center(cs):
  #g = extract_ground(cs, skylimit) # rows having road, sidewalk or terrain below the sky
  #groundrows = list(filter(lambda x: x>=skylimit, (np.where((g).sum(axis=1)>0)[0])))
  """
  if len(groundrows)==0:
    py = skylimit + int(random.random()*(cs.shape[0]-skylimit))
    px = int(random.random()*cs.shape[1])
  else:
    py = random.choice(groundrows) # random row that has ground groundrows[0]
    px = random.choice(np.where(g[py])[0]) # random ground pixel in that row
  """
  py = random.choice(list(range(max(int(random.random()*400),cs[0]//2),cs[0]+1)))
  px = int(random.random()*cs[1])
  return px, py

In [ ]:
def scaled_foreground(fg, newsize, py):
  fh, fw = fg.height, fg.width
  # get area wrt bg size
  r = min(random.random()+0.1, 0.5)
  csize = (newsize[0]*r, newsize[1]*r)
  ar = csize[0]*csize[1]

  # resize based on area and aspect
  dw = int(math.sqrt(ar)*fw/fh)
  dh = int(dw*(fh/fw))
  fg = fg.resize((dw, dh))

  # get mask of resized
  mask = np.array(fg)[:, :, 3]
  mask[mask<200]=0
  mask = Image.fromarray(mask, mode='L')

  return fg, mask

def place(bg, fg, mask, px, py):
  fh, fw = fg.height, fg.width
  py -= fh//2
  px -= fw//2
  bg.paste(fg, (px, py), mask)
  fmask = Image.new('1', bg.size)
  fmask.paste(mask, (px, py))

  return bg, fmask

In [ ]:
def flipforeground(fgpath, imgno):
  fglist = list(filter(lambda x: x.endswith('png'), os.listdir(fgpath)))
  print('Creating flipped foregrounds')
  for i in range(len(fglist)):
    fgimg = os.path.join(fgpath, fglist[i])
    fgim = Image.open(fgimg)
    fgout = fgim.transpose(Image.FLIP_LEFT_RIGHT)
    fgout.save(os.path.join(fgpath, f"image{imgno:0>3}.png"))
    imgno +=1

#We have 100 foreground images. the flipped images numbers will start from 101
#flipforeground('/content/drive/My Drive/EVA4/data/foreground', imgno=101)

In [ ]:
# from PIL import Image
# import numpy as np
# import osqp
# fgpath = os.path.join('/content/drive/My Drive/EVA4/data', 'foreground')

# for f in os.listdir():
#   rgba = np.array(Image.open(f).convert('RGBA'))
#   rgba[rgba[..., -1]>0] = [255,255,255,255]
#   rgba[rgba[..., -1]==0] = [0, 0,0, 255]
#   im = (Image.fromarray(rgba)).convert('RGB')
#   im.save('/content/drive/My Drive/EVA4/data/masks/mask'+f[5:-4]+'.jpg')

In [ ]:
#path for background and foreground images
bgpath = os.path.join('/content/drive/MyDrive/QuMaDe2', "bgImages")
fgpath = os.path.join('/content/drive/MyDrive/QuMaDe2', "fgImages")
segpath = os.path.join('/content/drive/MyDrive/QuMaDe2', "seg2")
#list of filenames of background images and foreground
bgimagelist = list(filter(lambda x: x.endswith('jpg'), os.listdir(bgpath)))
fgimagelist = list(filter(lambda x: x.endswith('png'), os.listdir(fgpath)))
seglist = list(filter(lambda x: x.endswith('npy'), os.listdir(segpath)))

In [ ]:
%tensorflow_version 1.x

In [ ]:
import os
import glob
import matplotlib
from keras.engine.topology import Layer, InputSpec
import keras.utils.conv_utils as conv_utils
import tensorflow as tf
import keras.backend as K

Using TensorFlow backend.


In [ ]:
class BilinearUpSampling2D(Layer):
    def __init__(self, size=(2, 2), data_format=None, **kwargs):
        super(BilinearUpSampling2D, self).__init__(**kwargs)
        self.data_format = K.normalize_data_format(data_format)
        self.size = conv_utils.normalize_tuple(size, 2, 'size')
        self.input_spec = InputSpec(ndim=4)

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            height = self.size[0] * input_shape[2] if input_shape[2] is not None else None
            width = self.size[1] * input_shape[3] if input_shape[3] is not None else None
            return (input_shape[0],
                    input_shape[1],
                    height,
                    width)
        elif self.data_format == 'channels_last':
            height = self.size[0] * input_shape[1] if input_shape[1] is not None else None
            width = self.size[1] * input_shape[2] if input_shape[2] is not None else None
            return (input_shape[0],
                    height,
                    width,
                    input_shape[3])

    def call(self, inputs):
        input_shape = K.shape(inputs)
        if self.data_format == 'channels_first':
            height = self.size[0] * input_shape[2] if input_shape[2] is not None else None
            width = self.size[1] * input_shape[3] if input_shape[3] is not None else None
        elif self.data_format == 'channels_last':
            height = self.size[0] * input_shape[1] if input_shape[1] is not None else None
            width = self.size[1] * input_shape[2] if input_shape[2] is not None else None
        
        return tf.image.resize_images(inputs, [height, width], method=tf.image.ResizeMethod.BILINEAR, align_corners=True)

    def get_config(self):
        config = {'size': self.size, 'data_format': self.data_format}
        base_config = super(BilinearUpSampling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:

import numpy as np
from PIL import Image

def DepthNorm(x, maxDepth):
    return maxDepth / x

def predict(model, images, minDepth=1, maxDepth=100, batch_size=2):
    # Support multiple RGBs, one RGB image, even grayscale 
    if len(images.shape) < 3: images = np.stack((images,images,images), axis=2)
    if len(images.shape) < 4: images = images.reshape((1, images.shape[0], images.shape[1], images.shape[2]))
    # Compute predictions
    model
    predictions = model.predict(images, batch_size=batch_size)
    # Put in expected range
    return np.clip(DepthNorm(predictions, maxDepth=maxDepth), minDepth, maxDepth) / maxDepth

def to_multichannel(i):
    if i.shape[2] == 3: 
      return i
    i = i[:,:,0]
    return np.stack((i,i,i), axis=2)

In [ ]:
# Keras / TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from matplotlib import pyplot as plt

In [ ]:
# Custom object needed for inference and training
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}

In [ ]:
print('Loading model...')
model='/content/drive/MyDrive/QuMaDe2/Utils/nyu.h5'
# Load model into GPU / CPU
model = load_model(model, custom_objects=custom_objects, compile=False)

print('\nModel loaded ({0}).'.format(model))

Loading model...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



Model loaded (<keras.engine.training.Model object at 0x7fc84054d9d0>).


In [ ]:
from PIL import Image
import numpy as np


def create_images(outputs, info, zipfolder):   
    for i in range(outputs.shape[0]):
      #print(info[i])
      newsize = (224,224)
      a = outputs[i].reshape(newsize).astype(np.float32)
      #print(type(a),a.shape)
      b = 1 - a
      m = np.max(b)
      mn = np.min(b)
      b = (b-mn)/(m-mn)
      b = (b*255).astype(np.uint8)
      img = Image.fromarray(b, mode='L')
      img = img.resize(newsize)
      n = info[i][0]
      #a = np.array(img).astype(np.uint8)
      #np.save(os.path.join(outpath, f"depth{n[0]}.npy"), a)
      #np.save("npytemp.npy", a)
      img.save("depthtemp.jpg")
      #img.save(os.path.join(outpath, f"depth{n[0]}.jpg"))
      zipfolder.write('depthtemp.jpg', f'output/depth/fgbg{n:0>6d}.jpg')
      #zipfolder.write('npytemp.npy', f'output/depth/fgbg{n:0>6d}.npy')

In [ ]:
# pipeline
# for a batch size of 2000 
# len(bg) = 1
# len(fg) = 100
# multiplier = 20
# this method saves fgonbg, mask, depth images on to their respective folders of the zipfolder (batch - 2000)
# This method need to be called twice for every bg...(ater testing for several batch sizes)
# original background images are 448X448, we are resizing fgonbg and mask to 224X224
# passing 448$448 to depth prediction and getting 224X224.
# All 1200 K images are of size 224 X 224

def run_one_batch(inpath, bg,seg, fglist, zipfolder, multiplier=20, imgno=0):
  loaded_images =[]
  imagemeta = []

  bgpath = os.path.join(inpath, "bgImages")
  fgpath = os.path.join(inpath, "fgImages")
 

  bgimg = os.path.join(bgpath, bg)
  original_im = Image.open(bgimg)
  w, h = original_im.width, original_im.height
  seg = np.load(inpath+"/seg2/"+seg)
  newsize = (224, 224) 
  bgsize = (448,448)
  

  for i in tqdm(range(len(fglist))):
    fgimg = os.path.join(fgpath, fglist[i])
    fg = Image.open(fgimg)

    for _ in range(multiplier):
      cropped = random_crop_n_flip(original_im,seg, bgsize)
      # Get the first row that is all non sky from top in the new image dimension
      #skylimit = int((np.argmax(np.sum((cs==10), axis=1)==0)))
      
      px, py = pick_random_center(bgsize)

      
      
      
      fg, mask = scaled_foreground(fg, newsize,py)
      finalimg, finalmask = place(cropped, fg, mask, px, py)
      imagemeta.append((imgno, finalimg.width, finalimg.height))
      x = np.clip(np.asarray(finalimg, dtype=float) / 255, 0, 1)

      finalimg.resize(newsize).save("bgtemp.jpg", optimize = True, quality=60)
      finalmask.resize(newsize).save("maskimg.jpg", optimize = True, quality=60)
      
      loaded_images.append(x)
      
      zipfolder.write('bgtemp.jpg', f'output/images/fgbg{imgno:0>6d}.jpg')
      zipfolder.write('maskimg.jpg', f'output/masks/mask{imgno:0>6d}.jpg')
      imgno += 1
  print("Generating Depth images .....")
  depthbatch = np.stack(loaded_images, axis=0)
  
  outputs = predict(model, depthbatch)
  
  create_images(outputs, imagemeta, zipfolder)
  
  return imgno


    
   # multiplex
   # create a numpy array
   # stack all 448x448 bg-fg images in that array, along size create 224x224 masks
   # pass that to depth network to get 224x224 depth numpy values
   # now resize 448x448 to 224x224 images and save in zipfolder/images
   # save masks in the masks zipfolder/folder
   # save depth images as well as .npy values in depths zipfolder/folder

# time for one batch, and see it should be ~5 sec but < 10sec

## Generating the Final data

In [ ]:
import shutil

In [ ]:
from zipfile import ZipFile


In [ ]:
# #overlaying fg on bg and generating masks for 25 background images
# imgno = 0
# for x in range(10):
#   print("background image - ", (x+1))
#   imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[20+x], fgimagelist[:50], zipfolder=out, multiplier=10, imgno=imgno)
#   #imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[20+x],fgimagelist[100:], zipfolder=out, multiplier=20, imgno=imgno)

# print("images created so far: ", imgno)

In [ ]:
#overlaying fg on bg and generating masks for 25 background images
from time import time
imgno = 0
for x in range(0,25):
  print("background image - ", (x+1))
  t1 = time()
  out = ZipFile('/content/drive/MyDrive/QuMaDe2/output/part1/fgbganddepth_part1.zip', mode='a')
  out.close()
  shutil.copy('/content/drive/MyDrive/QuMaDe2/output/part1/fgbganddepth_part1.zip','/content')
  out = ZipFile('/content/fgbganddepth_part1.zip',mode='a')
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x],seglist[x], fgimagelist[:50], zipfolder=out, multiplier=20, imgno=imgno)
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x], seglist[x], fgimagelist[50:100], zipfolder=out, multiplier=20, imgno=imgno)
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x], seglist[x], fgimagelist[100:150], zipfolder=out, multiplier=20, imgno=imgno)
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x], seglist[x], fgimagelist[150:200], zipfolder=out, multiplier=20, imgno=imgno)
  out.close()
  shutil.copy('/content/fgbganddepth_part1.zip','/content/drive/MyDrive/QuMaDe2/output/part1')
  print(time()-t1,"seconds for one image")


print("images created so far: ", imgno)

In [ ]:
#overlaying fg on bg and generating masks for 25 background images
from time import time
for x in range(25,50):
  print("background image - ", (x+1))
  t1 = time()
  out = ZipFile('/content/drive/MyDrive/QuMaDe2/output/part2/fgbganddepth_part2.zip', mode='a')
  out.close()
  shutil.copy('/content/drive/MyDrive/QuMaDe2/output/part2/fgbganddepth_part2.zip','/content')
  out = ZipFile('/content/fgbganddepth_part2.zip',mode='a')
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x],seglist[x], fgimagelist[:50], zipfolder=out, multiplier=20, imgno=imgno)
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x], seglist[x], fgimagelist[50:100], zipfolder=out, multiplier=20, imgno=imgno)
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x], seglist[x], fgimagelist[100:150], zipfolder=out, multiplier=20, imgno=imgno)
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x], seglist[x], fgimagelist[150:200], zipfolder=out, multiplier=20, imgno=imgno)
  out.close()
  shutil.copy('/content/fgbganddepth_part2.zip','/content/drive/MyDrive/QuMaDe2/output/part2')
  print(time()-t1,"seconds for one image")


print("images created so far: ", imgno)

In [ ]:
#overlaying fg on bg and generating masks for 25 background images
from time import time
for x in range(50,75):
  print("background image - ", (x+1))
  t1 = time()
  out = ZipFile('/content/drive/MyDrive/QuMaDe2/output/part3/fgbganddepth_part3.zip', mode='a')
  out.close()
  shutil.copy('/content/drive/MyDrive/QuMaDe2/output/part3/fgbganddepth_part3.zip','/content')
  out = ZipFile('/content/fgbganddepth_part3.zip',mode='a')
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x],seglist[x], fgimagelist[:50], zipfolder=out, multiplier=20, imgno=imgno)
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x], seglist[x], fgimagelist[50:100], zipfolder=out, multiplier=20, imgno=imgno)
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x], seglist[x], fgimagelist[100:150], zipfolder=out, multiplier=20, imgno=imgno)
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x], seglist[x], fgimagelist[150:200], zipfolder=out, multiplier=20, imgno=imgno)
  out.close()
  shutil.copy('/content/fgbganddepth_part3.zip','/content/drive/MyDrive/QuMaDe2/output/part3')
  print(time()-t1,"seconds for one image")


print("images created so far: ", imgno)

background image -  70


  0%|          | 0/50 [00:00<?, ?it/s]

Generating Depth images .....



  0%|          | 0/50 [00:00<?, ?it/s]

Generating Depth images .....


...

Generating Depth images .....


  0%|          | 0/50 [00:00<?, ?it/s]

Generating Depth images .....


  0%|          | 0/50 [00:00<?, ?it/s]

Generating Depth images .....
835.5809659957886 seconds for one image
images created so far:  500000


In [ ]:
#overlaying fg on bg and generating masks for 25 background images
from time import time
imgno = 0
for x in range(75, 100):
  print("background image - ", (x+1))
  t1 = time()
  out = ZipFile('/content/drive/MyDrive/QuMaDe2/output/part4/fgbganddepth_part4.zip', mode='a')
  out.close()
  shutil.copy('/content/drive/MyDrive/QuMaDe2/output/part4/fgbganddepth_part4.zip','/content')
  out = ZipFile('/content/fgbganddepth_part1.zip',mode='a')
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x],seglist[x], fgimagelist[:50], zipfolder=out, multiplier=20, imgno=imgno)
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x], seglist[x], fgimagelist[50:100], zipfolder=out, multiplier=20, imgno=imgno)
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x], seglist[x], fgimagelist[100:150], zipfolder=out, multiplier=20, imgno=imgno)
  imgno = run_one_batch('/content/drive/MyDrive/QuMaDe2', bgimagelist[x], seglist[x], fgimagelist[150:200], zipfolder=out, multiplier=20, imgno=imgno)
  out.close()
  shutil.copy('/content/fgbganddepth_part4.zip','/content/drive/MyDrive/QuMaDe2/output/part4')
  print(time()-t1,"seconds for one image")


print("images created so far: ", imgno)

In [ ]:
"""
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button").click() 
}setInterval(ClickConnect,360000)
"""

'\nfunction ClickConnect(){\nconsole.log("Working"); \ndocument.querySelector("colab-toolbar-button").click() \n}setInterval(ClickConnect,360000)\n'